In [3]:
import psycopg2 as pg
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
connection = pg.connect(dbname = 'postgres',
                       host = 'localhost',
                       user = 'postgres',
                       password = 'admin',
                       port = 5432)
cursor = connection.cursor()

In [20]:
connection.rollback()

# Схема БД

![](images/db_model.png)

Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.

In [4]:
cursor.execute('''select distinct brand from products
                  where standard_cost > 1500''')
pd.DataFrame(cursor.fetchall()) # columns=['transaction_id','product_id','customer_id','transaction_date','online_order','order_status']

,0
0,Giant Bicycles
1,OHM Cycles
2,Solex
3,Trek Bicycles


Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.

In [20]:
cursor.execute('''select * from transaction
                  where order_status like 'Approved'
                  and transaction_date between '2017-04-01' and '2017-04-09' ''')
pd.DataFrame(cursor.fetchall()).shape

(531, 6)

Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [22]:
cursor.execute('''select distinct job_title from jobs
                  where job_industry_category in ('IT', 'Financial Services')
                  and job_title like 'Senior%'
                 ''')
pd.DataFrame(cursor.fetchall())

,0
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [8]:
cursor.execute('''select distinct brand from products prod
                  inner join transaction tr on prod.product_id = tr.product_id
                  inner join customer cus on cus.customer_id = tr.customer_id
                  inner join jobs on jobs.job_id = cus.job_id
                  where job_industry_category = 'Financial Services'
                 ''')
pd.DataFrame(cursor.fetchall())

,0
0,OHM Cycles
1,Trek Bicycles
2,WeareA2B
3,Solex
4,Norco Bicycles
5,Giant Bicycles


Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.

In [15]:
cursor.execute('''select customer.customer_id, first_name, last_name from customer
                  inner join transaction tr on tr.customer_id = customer.customer_id
                  inner join products on products.product_id = tr.product_id
                  where brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
                  and online_order = true
                  limit 10
                 ''')
pd.DataFrame(cursor.fetchall())

,0,1,2
0,3120,Lauree,O`Donnell
1,787,Norma,Batrim
2,2339,Damien,Haddeston
3,1542,Sanders,Fernyhough
4,3262,Maynord,L`Episcopi
5,3002,Tracey,Verdun
6,3368,Tanya,Boddis
7,2448,Lorri,Iston
8,1305,Averil,Gentiry
9,3455,Coop,Skey


Вывести всех клиентов, у которых нет транзакций.

In [17]:
cursor.execute('''select customer.customer_id, first_name, last_name from customer
                  left join transaction tr on tr.customer_id = customer.customer_id
                  where tr.transaction_id is null
                 ''')
pd.DataFrame(cursor.fetchall())

,0,1,2
0,3565,Charyl,Pottiphar
1,3535,Bren,Dabbes
2,3647,Carlyle,Frape
3,3903,Dayna,Cawthera
4,3519,Aldus,Kenningley
...,...,...,...
502,3811,Etta,Leele
503,3648,Tracie,Bartolomeoni
504,4000,Kippy,Oldland
505,3719,Dorette,Attridge


Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.

In [29]:
cursor.execute('''select distinct cus.customer_id, first_name, last_name
                  from customer cus
                  inner join transaction tr on cus.customer_id = tr.customer_id
                  inner join products prod on prod.product_id = tr.product_id
                  inner join jobs on jobs.job_id = cus.job_id
                  where job_industry_category = 'IT'
                  and standard_cost in (select max(standard_cost) from products
					   group by brand)
                 ''')
pd.DataFrame(cursor.fetchall())

,0,1,2
0,83,Kerr,Simmell
1,249,D`arcy,Slay
2,267,Edgar,Buckler
3,308,Iver,Dobsons
4,441,Felicle,Beneze
5,451,Marlow,Flowerdew
6,680,Gay,Pickersgill
7,769,Cameron,Letherbury
8,784,Frederik,Duckett
9,799,Harland,Spilisy


Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

In [34]:
cursor.execute('''select distinct cus.customer_id, first_name, last_name
                  from customer cus
                  inner join transaction tr on cus.customer_id = tr.customer_id
                  inner join products prod on prod.product_id = tr.product_id
                  inner join jobs on jobs.job_id = cus.job_id
                  where job_industry_category in ('IT', 'Health')
                  and cus.customer_id in (select customer_id from transaction
                                          where order_status = 'Approved'
                                          and transaction_date between '2017-07-07' and '2017-07-17')
                 ''')
pd.DataFrame(cursor.fetchall())

,0,1,2
0,1907,Lind,Dredge
1,685,Booth,Birkin
2,2337,Odilia,Bick
3,2341,Caterina,Scedall
4,1237,Junia,Humerstone
...,...,...,...
110,708,Idalina,Wilcot
111,788,Gregg,Townsend
112,694,Lilith,Lanning
113,2782,Gillian,Scrimshire


In [27]:
connection.rollback()